In [1]:
import time
import re
import board
import adafruit_dht
import adafruit_bmp280
from gpiozero import LED, Button
from llama_cpp import Llama

In [2]:
# Hardware Configuration
DHT_PIN = board.D16
LED_PINS = {'red': 13, 'yellow': 19, 'green': 26}
BUTTON_PIN = 20
BMP280_ADDRESS = 0x76
REAL_ALTITUDE = 960  # Adjust to your location

In [3]:
# Model Configuration
MODEL_PATH = "/home/mjrovai/models/models/original/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"

In [4]:
# Initialize Hardware
def init_hardware():
    # Sensors
    dht = adafruit_dht.DHT22(DHT_PIN)
    i2c = board.I2C()
    bmp = adafruit_bmp280.Adafruit_BMP280_I2C(i2c, address=BMP280_ADDRESS)
    bmp.sea_level_pressure = 1013.25
    
    # Actuators
    leds = {color: LED(pin) for color, pin in LED_PINS.items()}
    button = Button(BUTTON_PIN)
    
    return dht, bmp, leds, button

# Initialize SLM with optimized settings
def init_slm():
    return Llama(
        model_path=MODEL_PATH,
        n_ctx=512,
        n_threads=4,
        n_gpu_layers=0,
        verbose=False
    )

In [5]:
# Sensor Reading with error handling
def read_sensors(dht, bmp):
    sensor_data = {}
    try:
        # BMP280 Readings
        sensor_data.update({
            'bmp_temp': round(bmp.temperature, 1),
            'pressure': round(bmp.pressure, 1),
            'altitude': round(bmp.altitude)
        })
    except Exception as e:
        print(f"BMP280 Error: {str(e)}")
    
    try:
        # DHT22 Readings
        sensor_data.update({
            'dht_temp': round(dht.temperature, 1),
            'humidity': round(dht.humidity, 1)
        })
    except RuntimeError as e:
        print(f"DHT22 Error: {str(e)}")
    
    return sensor_data

In [6]:
# Command Processing Core
VALID_ACTIONS = {
    'turn_on_red', 'turn_off_red',
    'turn_on_yellow', 'turn_off_yellow',
    'turn_on_green', 'turn_off_green',
    'turn_on_all', 'turn_off_all',
    'read_sensors'
}

def process_command(llm, command):
    prompt = f"""<INST>
    Convert this command to ONE action keyword: "{command}"
    Valid actions: {sorted(VALID_ACTIONS)}
    Answer ONLY with the action keyword. No explanations.
    </INST>"""
    
    response = llm(prompt, max_tokens=1, temperature=0.0)
    raw_action = response['choices'][0]['text'].strip().lower()
    
    # Strict validation
    for action in VALID_ACTIONS:
        if action in raw_action:
            return action
    return 'unknown'

# LED Control Functions
def control_leds(leds, action):
    if action == 'turn_on_all':
        for led in leds.values():
            led.on()
    elif action == 'turn_off_all':
        for led in leds.values():
            led.off()
    else:
        color = action.split('_')[-1]
        if 'on' in action:
            leds[color].on()
        else:
            leds[color].off()

In [7]:
# Main Interaction Loop
def main():
    dht, bmp, leds, button = init_hardware()
    llm = init_slm()
    
    print("\n=== SLM Physical Computing Interface ===")
    print("Supported commands:")
    print("- 'read_sensors' : Show sensor data")
    print("- 'turn_on/off_[red|yellow|green|all]' : Control LEDs")
    print("- Press physical button for instant sensor read")
    print("- Type 'exit' to quit\n")
    
    try:
        while True:
            # Button-triggered sensor read
            if button.is_pressed:
                data = read_sensors(dht, bmp)
                print("\n=== Button-triggered Sensor Read ===")
                print(f"Temperature: {data.get('dht_temp', 'N/A')}°C")
                print(f"Humidity: {data.get('humidity', 'N/A')}%")
                print(f"Pressure: {data.get('pressure', 'N/A')} hPa")
                print(f"Altitude: {data.get('altitude', 'N/A')} m")
                time.sleep(1)  # Debounce
            
            # Process text commands
            command = input("\nSLM> ").strip().lower()
            if not command:
                continue
            if command == 'exit':
                break
                
            action = process_command(llm, command)
            
            if action == 'unknown':
                print(f"Could not understand: {command}")
            elif action == 'read_sensors':
                data = read_sensors(dht, bmp)
                print("\n=== Sensor Data ===")
                print(f"Temperature (DHT): {data.get('dht_temp', 'N/A')}°C")
                print(f"Humidity: {data.get('humidity', 'N/A')}%")
                print(f"Temperature (BMP): {data.get('bmp_temp', 'N/A')}°C")
                print(f"Pressure: {data.get('pressure', 'N/A')} hPa")
            else:
                control_leds(leds, action)
                print(f"Action: {action.replace('_', ' ').title()}")
                
    finally:
        # Cleanup
        for led in leds.values():
            led.off()
        print("\nSystem shutdown. LEDs turned off.")

In [10]:
main()

Unable to set line 16 to input


GPIOPinInUse: pin GPIO13 is already in use by <gpiozero.LED object on pin GPIO13, active_high=True, is_active=False>